In [24]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
from PIL import Image
import os
print(os.listdir("../input/cell-images-for-detecting-malaria/cell_images/cell_images"))


['Uninfected', 'Parasitized']


In [26]:
infected_data=os.listdir("../input/cell-images-for-detecting-malaria/cell_images/cell_images/Parasitized")
uninfected_data=os.listdir("../input/cell-images-for-detecting-malaria/cell_images/cell_images/Uninfected")
print(len(infected_data))
print(len(uninfected_data))

13780
13780


In [27]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.3),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [28]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

In [29]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1/255.0, validation_split=0.2)
trainDatagen = datagen.flow_from_directory(directory='../input/cell-images-for-detecting-malaria/cell_images/cell_images/',
                                           target_size=(128,128),
                                           class_mode = 'binary',
                                           batch_size = 16,
                                           subset='training')

Found 22048 images belonging to 2 classes.


In [30]:
validationDatagen = datagen.flow_from_directory(directory='../input/cell-images-for-detecting-malaria/cell_images/cell_images/',
                                           target_size=(128,128),
                                           class_mode = 'binary',
                                           batch_size = 16,
                                           subset='validation')

Found 5510 images belonging to 2 classes.


In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 126, 126, 16)      448       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 63, 63, 16)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 63, 63, 16)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 61, 61, 32)        4640      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 28, 28, 64)       

In [32]:
class mycallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epochs,logs={}):
        if(logs.get('accuracy')>0.98):
            self.model.stop_training=True

In [ ]:
callbacks=mycallbacks()
history=model.fit_generator(trainDatagen,validation_data=validationDatagen,epochs=10,verbose=1,callbacks=[callbacks])

Epoch 1/10
1378/1378 [==============================] - 237s 172ms/step - loss: 0.6123 - accuracy: 0.6520 - val_loss: 0.4475 - val_accuracy: 0.8287
Epoch 2/10
1378/1378 [==============================] - 230s 167ms/step - loss: 0.2725 - accuracy: 0.8976 - val_loss: 0.2010 - val_accuracy: 0.9403
Epoch 3/10
1378/1378 [==============================] - 235s 171ms/step - loss: 0.1779 - accuracy: 0.9485 - val_loss: 0.1950 - val_accuracy: 0.9445
Epoch 4/10
1378/1378 [==============================] - 234s 170ms/step - loss: 0.1563 - accuracy: 0.9547 - val_loss: 0.1783 - val_accuracy: 0.9468
Epoch 5/10
1378/1378 [==============================] - 237s 172ms/step - loss: 0.1480 - accuracy: 0.9559 - val_loss: 0.1712 - val_accuracy: 0.9485
Epoch 6/10
1378/1378 [==============================] - 233s 169ms/step - loss: 0.1434 - accuracy: 0.9569 - val_loss: 0.1743 - val_accuracy: 0.9495
Epoch 7/10
1378/1378 [==============================] - 236s 172ms/step - loss: 0.1396 - accuracy: 0.9580 - val_

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()